# Practical Python Programming for Biologists
Author: Dr. Daniel Pass | www.CompassBioinformatics.com

---

# Day 3 Project Challenge - Covid dataset with modules

In this exercise we are going to look at a complex dataset (n=126) of covid S protein reference sequences and associated metadata and statistics, and we're going to use some 3rd party modules to perform some higher-level analysis.

As we have seen today there are many (many!) ways to approach any dataset. First consider a question that you want to test from this data. If you are more interested in statistical testing then you can focus on the metadata and statistics with pandas/scipy. If you are more interested in sequence bioinformatics then you can work with the fasta data. However you'll likely need both to fully investigate.

You have been provided with two files:
- ```Day3-Project-nextclade_metadata``` - Data & Metadata of 126 reference covid19 strains and the Spike (S) protein modificiation data
- ```Day3-Project-nextclade.fasta``` - Reference fasta sequence file for the corresponding spike proteins

Essential steps:
1. Read in and filter the metadata file to remove bad and missing data (pandas!)
2. Read in the fasta sequence file using biopython. Only include sequences that have passed your step one filter!
3. Now perform some analysis!

Some example challenges:
- A function to output the spike protein sequence and key statistics (GC%, length, etc)
- Calculate the number/size of gap regions (strings of Ns) - does this correspond with the quality assesment?
- A statistical test for if binding efficiency correlates with any other variables
- Test variations in sequence substitutions in metadata
- Calculate summary statistics per clade/strain
- Output a phylogenetic tree of strains (I think the full dataset may be too computaionally intensive for google Colab, however you could use one/two sequences per clade, or alternatively your own PCs)

This project is largely undefined and the intention is to give you a real-life look at approaching a dataset for the first time. Think about your objectives before jumping in! Good luck!


### Some statistics

In [16]:
import pandas as pd

## Read in metadata
df = pd.read_csv("C:/Users/danie/Google Drive/Work/CompassBioinformatics/Courses/IntroToPython/ClassData/Day3-Project-nextclade_metadata.csv")

print(df.columns)
print(df.head())

Index(['index', 'seqName', 'clade', 'Nextclade_pango', 'partiallyAliased',
       'clade_nextstrain', 'clade_who', 'clade_legacy', 'qc.overallScore',
       'qc.overallStatus', 'totalSubstitutions', 'totalDeletions',
       'totalInsertions', 'totalFrameShifts', 'totalMissing', 'totalNonACGTNs',
       'totalAminoacidSubstitutions', 'totalAminoacidDeletions',
       'totalAminoacidInsertions', 'totalUnknownAa', 'alignmentScore',
       'alignmentStart', 'alignmentEnd', 'coverage', 'substitutions',
       'bindingEfficiency'],
      dtype='object')
   index                                            seqName        clade  \
0      0                                           OX458355          23B   
1      1                                      IND/2930/2023          22F   
2      2  IMS-10116-CVDP-88333393-399B-4563-A7B1-14DFC5D...          21L   
3      3  IMS-10004-CVDP-63A4C3D8-D0DC-4A57-93D5-BB10D58...          22D   
4      4                         England/QEUH-3267E648/2022  recom

In [18]:
goodSamples = df[df["qc.overallStatus"] == "good"]

print(goodSamples.groupby("clade_who")["totalAminoacidSubstitutions"].mean())

clade_who
Delta          30.500000
Epsilon        14.333333
Eta            19.000000
Kappa          21.333333
Lambda         21.000000
Mu             23.666667
Omicron        57.452830
Theta          20.000000
recombinant    45.571429
Name: totalAminoacidSubstitutions, dtype: float64


In [32]:
from scipy.stats import mannwhitneyu, spearmanr

corrs_to_test = ['totalAminoacidSubstitutions', 'totalAminoacidDeletions','totalAminoacidInsertions']

# All samples
for x in corrs_to_test:
    stat, pvalue = spearmanr(df[x], df["bindingEfficiency"])
    print("Spearman Rank test", x, "vs binding Efficiency")
    print("R2= {} (p={})".format(round(stat,3), round(pvalue, 5)))

print("================")
# 'good' samples
for x in corrs_to_test:
    stat, pvalue = spearmanr(goodSamples[x], goodSamples["bindingEfficiency"])
    print("Spearman Rank test", x, "vs binding Efficiency")
    print("R2= {} (p={})".format(round(stat,3), round(pvalue, 5)))



Spearman Rank test totalAminoacidSubstitutions vs binding Efficiency
R2= 0.974 (p=0.0)
Spearman Rank test totalAminoacidDeletions vs binding Efficiency
R2= 0.769 (p=0.0)
Spearman Rank test totalAminoacidInsertions vs binding Efficiency
R2= 0.032 (p=0.65916)
Spearman Rank test totalAminoacidSubstitutions vs binding Efficiency
R2= 0.985 (p=0.0)
Spearman Rank test totalAminoacidDeletions vs binding Efficiency
R2= 0.735 (p=0.0)
Spearman Rank test totalAminoacidInsertions vs binding Efficiency
R2= 0.073 (p=0.39732)


### loading sequence data

In [33]:
# Read in the fasta file as a biopython object
from Bio import SeqIO, Seq

# Read in the fasta file as a biopython object
strain_sequences = []
for seq_record in SeqIO.parse("C:/Users/danie/Google Drive/Work/CompassBioinformatics/Courses/IntroToPython/ClassData/Day3-Project-nextclade.fasta", "fasta"):
  strain_sequences.append(seq_record)

#### Functions

In [43]:
def GC_pc(seq_record):
    sequence = seq_record.seq
    GC_pc = (sequence.count("G") + sequence.count("C")) / len(sequence) *100
    return round(GC_pc,2)

def number_of_gaps(seq_record, min_size):
    sequence = seq_record.seq
    N_sizes = []
    counter = 0
    for base in sequence:
        if base == "N":
            counter += 1
        else:
            if counter > min_size:
                N_sizes.append(counter)
            counter = 0

    return len(N_sizes)


findings = {}
for record in strain_sequences:
    #print(record.id)
    findings[record.id] = [GC_pc(record), number_of_gaps(record, 3)]
    #print(f"{record.id} \t {GC_pc(record)}% \t {number_of_gaps(record, 3)}")


for ID in findings.keys():
    GC, gaps = findings.get(ID)
    seq_qual = df.loc[df['seqName'] == ID, 'qc.overallStatus'].values
    if gaps >5:
        print("GC: {}%\tNo. gaps: {}\t{}\t{}".format(GC, gaps, seq_qual, ID))

GC: 36.26%	No. gaps: 6	['good' 'good' 'good' 'good' 'good']	OX458355
GC: 37.48%	No. gaps: 11	['good' 'good' 'good']	IND/2930/2023
GC: 36.74%	No. gaps: 9	[]	IMS10004CVDP63A4C3D8D0DC4A5793D5BB10D58E8487
GC: 37.79%	No. gaps: 7	['good']	OP958840
GC: 34.26%	No. gaps: 12	['mediocre']	ON629031
GC: 34.81%	No. gaps: 158	[]	USA/TXCDCASC210279141/2021
GC: 11.75%	No. gaps: 20	['bad']	LR991190
GC: 29.97%	No. gaps: 90	[]	USA/COCDPHE2007261261/2020
GC: 37.53%	No. gaps: 19	['good']	USA/TG278209/2020
GC: 34.8%	No. gaps: 7	['unknown']	SYC/G38725/2021
GC: 36.71%	No. gaps: 7	['unknown']	OU221273
GC: 37.32%	No. gaps: 8	[]	USA/NJPHEL2112068/2021
GC: 35.64%	No. gaps: 9	[]	USA/WAS5413/2021
GC: 37.23%	No. gaps: 12	[]	USA/IDCDCASC210011921/2021
GC: 36.17%	No. gaps: 13	[]	USA/VACDCLC0067096/2021
GC: 23.03%	No. gaps: 47	['bad' 'bad']	BHR/30005866/2021
GC: 36.12%	No. gaps: 30	[]	USA/CACDPH3000131458/2021
GC: 37.14%	No. gaps: 7	[]	USA/MACDCBICRSP_6XNVDTWZVHP437HT/2021
GC: 37.52%	No. gaps: 14	[]	USA/CACDCFG042304/20